In [ ]:
SSH_KEY_LOCATION = "D:\ArkhoTech\CCHC\pem.pem"

from sshtunnel import SSHTunnelForwarder
import pandas as pd
import awswrangler as wr
import pyodbc

HOST = "ec2-54-152-91-151.compute-1.amazonaws.com"
USER = "ec2-user"
PORT = 22

def getConnection(port):
    con_string = f"""DRIVER={{ODBC Driver 17 for SQL Server}};SERVER=127.0.0.1,{port};DATABASE=bne;UID=admin;PWD=ArkhoBN3"""
    conn = pyodbc.connect(con_string)
    return conn

def query(query_string: str):
    with SSHTunnelForwarder(
            (HOST, PORT),
            ssh_username=USER,
            ssh_pkey=SSH_KEY_LOCATION,
            remote_bind_address=("bne.cppgzemeawv9.us-east-1.rds.amazonaws.com", 1433),
        ) as server:
            conn = getConnection(server.local_bind_port)
            return pd.read_sql(query_string, con=conn)


In [ ]:
df = query("""
select
       convert(varchar(10), F.FEC_ALTA, 120) AS fecha_alta,
       BNE_COMUNAS.NOMBRE                    AS comuna,
       BNE_REGIONES.NOMBRE                   AS region,
       COUNT(*)                              AS total,
        co.DESCRIPCION as ocupacion
from EMP_EMPRESARIO_USUARIOS a
         inner join EMP_USUARIOS_UBICAC b on b.ID_EMPRESARIO_USUARIOS = a.id
         inner join EMP_UBICAC_RELAC c on c.id = b.id_ubic_rel
         inner join emp_empresario d on d.id = c.id_empresario
         inner join sct_emp_sectores e on e.ID_EMPRESARIO = d.id
         inner JOIN dbo.CCHC_DESCARGA_CV F ON a.id = f.id_emp_usuario
         LEFT JOIN BNE_COMUNAS ON BNE_COMUNAS.ID = d.ID_COMUNA
         LEFT JOIN BNE_REGIONES ON BNE_REGIONES.ID = d.ID_REGION
        LEFT JOIN CCHC_DESCARGA_CV_DETALLE CDCD ON CDCD.ID_DESCARGA_CV = F.ID
        LEFT JOIN PER_PERSONAS pp ON pp.id = CDCD.ID_PERSONA
        LEFT JOIN CCHC_PER_OCUPACIONES cpo ON cpo.ID_PERSONA = pp.ID
        LEFT JOIN CCHC_OCUPACIONES co ON co.id = cpo.ID_OCUPACION
GROUP BY convert(varchar(10), F.FEC_ALTA, 120), BNE_COMUNAS.NOMBRE, BNE_REGIONES.NOMBRE, DESCRIPCION;
""")

df.head()

In [ ]:
df["fecha_alta"] = pd.to_datetime(df.fecha_alta)

df.head()

In [ ]:
wr.s3.to_parquet(
    df=df,
    database="raw_dev",
    table="bne_oficios_cvs_descargados",
    path="s3://cchc-dw-dev-raw/bne/oficios_cvs_descargados/",
    mode="overwrite",
    dtype={
        "fecha_alta" : "date"
    },
    dataset=True
)

In [ ]:
# Punto 2

df = query(
    """
SELECT
    CONCAT(per.NUM_DOCUMENTO, '-', per.DIGITO_VERIFICADOR) as rut,
    regiones.NOMBRE as region,
    comunas.NOMBRE as comuna,
    COUNT(media.ID  ) as ed_media,
    COUNT(cursos.ID) as cursos,
    COUNT(superior.ID ) as ed_superior
FROM dbo.PER_PERSONAS per
LEFT JOIN dbo.CCHC_PERSONA_EMTP media ON media.ID_PERSONA = per.ID
LEFT JOIN dbo.CCHC_PERSONA_CURSO cursos ON cursos.ID_PERSONA = per.ID
LEFT JOIN dbo.CCHC_PERSONA_ETESUP superior ON superior.ID_PERSONA = per.ID
LEFT JOIN dbo.BNE_REGIONES regiones ON regiones.id = per.ID_REGION
LEFT JOIN dbo.BNE_COMUNAS comunas ON comunas.id = per.ID_COMUNA
GROUP BY CONCAT(per.NUM_DOCUMENTO, '-', per.DIGITO_VERIFICADOR), regiones.NOMBRE, comunas.NOMBRE
HAVING COUNT(media.ID  ) > 0
    OR COUNT(cursos.ID) > 0
    OR COUNT(superior.ID ) > 0;
"""
)

df.head()


In [ ]:
geografia = wr.s3.read_parquet("s3://cchc-dw-qa-analytics/geografia_variaciones/",columns=['region','comuna'])
geografia

In [ ]:
##staging
df2 = df.copy()

df2.loc[df2.ed_media > 0, "tiene_ed_media"] = True
df2.loc[df2.cursos > 0, "tiene_cursos"] = True
df2.loc[df2.ed_superior > 0, "tiene_ed_superior"] = True

df2.fillna({
    "tiene_ed_media" : False,
    "tiene_cursos" : False,
    "tiene_ed_superior" : False,
}, inplace=True)

In [ ]:
import unicodedata


def convertir_ascii(campo):
    if campo is None:
        return None

    campo = unicodedata.normalize('NFKD', campo).encode('ascii', 'ignore').decode('UTF-8').lower()
    return campo


dic = {
"Los Lagos":"Los Lagos",
 "Valparaíso":"Valparaíso",
 "Metropolitana":"Metropolitana de Santiago",
 "O'Higgins": "Libertador General Bernardo O'Higgins",
 "Antofagasta":"Antofagasta",
 "Maule":"Maule",
 "La Araucanía":"La Araucanía",
 "Coquimbo":"Coquimbo",
 "Bío Bío":"Biobío",
 "Atacama":"Atacama",
 "Los Ríos":"Los Ríos",
 "Arica y Parinacota":"Arica y Parinacota",
 "Tarapacá":"Tarapacá",
 "Magallanes y Antártica Chilena":"Magallanes y de la Antártica Chilena",
 "Ñuble":"Ñuble",
 "Aysén": "Aysén del General Carlos Ibáñez del Campo"}
 


###ACA ABAJO ANALYTICS

dfsinComuna = df2[(df2.comuna.isnull()) & (df2.region != 'MIGRACION')]
dfComuna = df2[(df2.comuna.notnull())]

dfComuna.drop(columns=['region'], inplace=True)
dfComuna['comunaAsci'] = dfComuna['comuna'].apply(lambda x: convertir_ascii(x))
geografia['comunaAsci'] = geografia['comuna'].apply(lambda x: convertir_ascii(x))

dfComuna = dfComuna.merge(geografia[['comunaAsci','region']], how='left', on='comunaAsci')


dfsinComuna['region'] = dfsinComuna.region.map(dic)

df2 = pd.concat([dfsinComuna,dfComuna])

df2.drop(columns=["comunaAsci","ed_media", "cursos", "ed_superior"], inplace=True)

df2





In [ ]:
wr.s3.to_parquet(
    df=df,
    database="raw_dev",
    table="bne_nivel_estudios",
    path="s3://cchc-dw-dev-raw/bne/bne_nivel_estudios/",
    mode="overwrite",
    dataset=True
)

In [ ]:
df = query('''SELECT COUNT(*) as CONTEO, EVALUACION FROM dbo.CCHC_EVALUACION
GROUP BY EVALUACION''')

df

In [ ]:
wr.s3.to_parquet(
    df=df,
    database="raw_dev",
    table="bne_evaluaciones",
    path="s3://cchc-dw-dev-raw/bne/bne_evaluaciones/",
    mode="overwrite",
    dataset=True
)